## Ch15 Ex2 Twspider

In [2]:
from urllib.request import urlopen
import urllib.error
import json
import sqlite3
import ssl
import twurl

In [3]:
TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

In [5]:
conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

In [7]:
cur.execute('''CREATE TABLE IF NOT EXISTS Twitter(name TEXT, retrieve INTEGER, freinds INTEGER)''');

In [9]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
# 보안 에러 무시코드

In [10]:
while True:
    acct = input()
    if (acct == 'quit'):
        break
    if(len(acct) < 1):
        cur.execute('''SELECT name FROM Twitter WHERE retrieve = 0 LIMIT 1''')
        try:
            acct = cur.fetchone()[0]
        except:
            print('No unretrieved Twitter accounts found')
            continue
    
    url = twurl.augment(TWITTER_URL, {'screen_name':acct, 'count':'5'})
    print('Retrieving', url)
    connection = urlopen(url, context = ctx)
    data = connection.read().decode()
    headers = dict(connection.getheaders())
    
    print('Remaining', headers['x-rate-limit-remaining'])
    js = json.loads(data)

hi
Retrieving https://api.twitter.com/1.1/friends/list.json?oauth_consumer_key=h7Lu...Ng&oauth_timestamp=1533475973&oauth_nonce=73183940&oauth_version=1.0&screen_name=hi&count=5&oauth_token=10185562-eibxCp9n2...P4GEQQOSGI&oauth_signature_method=HMAC-SHA1&oauth_signature=MMKONDpb0PbbmR7R3fvyWT%2F0uwQ%3D


HTTPError: HTTP Error 401: Authorization Required

In [11]:
cur.execute('UPDATE Twitter SET retrieved = 1 WHERE name = ?', (acct,))

OperationalError: no such column: retrieved

In [12]:
countnew = 0
countold = 0
for u in js['users']:
    friend = u['screen_name']
    print(friend)
    cur.execute('SELECT freinds FROM Twitter WHERE name = ? LIMIT 1', (friend,))
    
    try:
        count = cur.fetchone()[0]
        cur.execute('UPDATE Twitter SET friends = ? WHERE name = ?', (count+1, friend))
        countold = countold + 1
    except:
        cur.execute('''INSERT INTO Twitter (name, retrieved, friends) VALUES(?, 0, 1)''', (friend,))
        countnew = countnew + 1
        
    print('New account = ', countnew, 'revisited = ',countold)
    conn.commit()

NameError: name 'js' is not defined

In [13]:
cur.close()